# pdf-scraper

In [1]:
from pathlib import Path
PROJECT_ROOT_DIR = Path.cwd().resolve()
PROJECT_ROOT_DIR

PosixPath('/Users/etjenahmic/personal-projects/pdf-scraper')

In [2]:
import pandas as pd

In [3]:
from loguru import logger
logger.add(PROJECT_ROOT_DIR / "pdf-scraper-log.txt", level="DEBUG", format='{time:HH:mm:ss} ({name}:{function}:{line}) - {message}')

1

In [4]:
import re
import pdfplumber
from pprint import pprint

with pdfplumber.open(PROJECT_ROOT_DIR / "factuur.pdf") as pdf:
    table = []
    for page in pdf.pages:
        t = page.extract_table()
        if t:
            table.extend(t)

def split_string(row: str) -> tuple[str]:
    pattern = re.compile(r'(\S+)\s+(\d+,\d{2})\s+(.+?)\s+(\d+,\d{2})\s+(\d+ %)\s+(\d+,\d{2})')
    match = re.match(pattern, row)
    assert len(match.groups()) == 6, logger.debug(f"{row}\nTuple length is not equal to 6.")
    return match.groups()

data = [split_string(row[0]) for row in table]

df = pd.DataFrame(data, columns=["Artikelnr", "Aantal", "Omschrijving", "Prijs per stuk", "Korting", "Regeltotaal"])
df

,Artikelnr,Aantal,Omschrijving,Prijs per stuk,Korting,Regeltotaal
0,104001,"1,00",Rol geperforeerd montageband l=10mtr,"55,28",45 %,"30,40"
1,90111,"4,00",Mtr komo buis 110 sn4 l=4mtr,"5,36",48 %,"11,15"
2,90103,"6,00",Mtr U3 komo buis 40x3,"1,78",48 %,"5,55"
3,101564,"2,00",Pvc lijmsifon 40 mm,"8,52",45 %,"9,37"
4,100270,"2,00",Inzet verloopstuk excentrisch 125x110,"2,99",45 %,"3,29"
5,101028,"2,00",Bocht 110x88 2xmof sn4 rubber,"7,80",45 %,"8,58"
6,400009,"1,00",Buitenrooster design rvs 125 mm,"42,87",42 %,"24,86"
7,706433,"7,00",Bonfix press bocht 22x90 staalverzinkt,"5,45",45 %,"20,98"
8,800511,"2,00",Tigris PPSU koppeling 25x22 mm,"26,19",45 %,"28,81"
9,701726,"3,00",Bonfix klem koppeling 15x1/2 buitdr,"3,64",45 %,"6,01"


In [5]:
df.to_excel("test.xlsx", index=False, engine="openpyxl")